### Import needed libraries 

In [1]:
#download the google map image from url
import urllib.request, requests, math

### Define functions that use Google Map APIs
All functions defined here require Google API Key

In [ ]:
# Checks if the given point (latitude, longitude) is within the city given 
# Returns true if the point is within the city, false otherwise
# Raises an exception if the Key given is invalid or you reached the query limit
# requires API key to have "Reverse Geocoding" API Library to be enabled
def is_within_city(latitude, longitude, city_name, key):
    url = "https://maps.googleapis.com/maps/api/geocode/json?latlng=" + str(latitude) + "," + str(longitude) + "&location_type=ROOFTOP|RANGE_INTERPOLATED|GEOMETRIC_CENTER&key=" + key
    response = requests.get(url)
    response.raise_for_status()
    if (response.json()['status'] == 'ZERO_RESULTS'):
        return False
    elif (response.json()['status'] != "OK"):
        raise BaseException("Something wrong happened when reverse-geocoding this location:", latitude , longitude, "This is the error received", response.json()['status'])

    address = response.json()["results"][0]["formatted_address"]
    city = address.split(",")[1].strip()
    return city == city_name

# downloads the satellite version of the image

def download_satelite_image(latitude, longitude, key, fileName, width=640, height=640, zoom=17):
    f = open(fileName, "wb")
    f.write(urllib.request.urlopen('https://maps.googleapis.com/maps/api/staticmap?center='+ str(latitude) +','+ str(longitude) + '&zoom=17&size=' +str(width) + 'x' + str(height) + '&maptype=satellite&key=' + key).read())
    f.close()
    
# downloads the road mpa version of the image
# width, height are the dimensions of the image produced
# zoom determines how far the image is zoomed in (bigger number = more zoomed in you are)
def download_map_image(latitude, longitude, key, fileName, width=640, height=640, zoom=17):
    f = open(fileName, "wb")
    f.write(urllib.request.urlopen('https://maps.googleapis.com/maps/api/staticmap?center=' + str(latitude) + ',' + str(longitude) + '&zoom=17&size=640x640&style=feature:landscape|element:geometry|saturation:-100&style=feature:water|saturation:-100|invert_lightness:true&style=element:labels|visibility:off&key=' + key).read())
    f.close()

### Define helper functions for coordinate and distance calculations 

In [4]:
# given a latitude, width of the picture in pixels, and the zoom level of the image
# returns the width of the pictue in meters 
# Code obtained from: https://gis.stackexchange.com/a/127949
def get_pic_width_meters(latitude, pic_width_pixels, zoom):
    meters_per_pixel = 156543.03392 * math.cos(latitude * math.pi / 180) / math.pow(2, zoom) 
    pic_width_meters = meters_per_pixel * pic_width_pixels
    return pic_width_meters


# get coordinates of 2nd point, given a starting point, distance moved, and either moving north/east/south/west
# latitude and longitude must be given in DEGREES
# returns coordinates of new point in DEGREES
def get_second_point(lat1, long1, distance, direction):
    ' direction: int: 1 = north, 2 = east, 3 = south, 4 = west'
    ret_value = {1: get_second_point_moving_NS(lat1, long1, distance, False),
                 2: get_second_point_moving_EW(lat1, long1, distance, False),
                 3: get_second_point_moving_NS(lat1, long1, distance, True),
                 4: get_second_point_moving_EW(lat1, long1, distance, True)
                 }
    return ret_value[direction]

# given starting coordinates, distance to travel, and whether moving east or west: return the coordinates of the destination
# uses the Haversine Formula (https://en.wikipedia.org/wiki/Haversine_formula#The_haversine_formula)
def get_second_point_moving_EW(lat1, long1, distance, west= True):
    radius = 6371000 # radius of earth in meters  
    lat1 = math.radians(lat1)
    long1 = math.radians(long1)
    p1 = math.sin(distance/(2*radius))
    p2 = math.cos(lat1)
    quotient = p1/p2
    delta_longitude = 2*math.asin(quotient)
    # print("inside fnctin", math.degrees(delta_longitude))
    # print()
    if west:
        long2 = long1 - delta_longitude
    else:
        long2 = long1 + delta_longitude
    lat2 = math.degrees(lat1) # doesn't change
    long2 = math.degrees(long2)
    return lat2, long2

# given starting coordinates, distance to travel, whether moving north or south: return the coordinates of the destination
# uses the Haversine Formula (https://en.wikipedia.org/wiki/Haversine_formula#The_haversine_formula)
def get_second_point_moving_NS(lat1, long1, distance, south=True):
    radius = 6371000  # radius of earth in meters
    lat1 = math.radians(lat1)
    long1 = math.radians(long1)
    delta_latitude = distance/radius
    if south:
        lat2 = lat1 - delta_latitude
    else:
        lat2 = lat1 + delta_latitude
    lat2 = math.degrees(lat2)
    long2 = math.degrees(long1) # doesn't change
    return lat2, long2